In [1]:
import numpy as np
import math
import random

In [2]:
#Initial Pheromone 
def FirstPheromone(hobs):
    return [[1 for i in range(hobs)] for j in range(hobs)]

# A random graph
def Graph(dim):
    
    b = [(i, str(i)) for i in range(dim)]   
    matrix = []
    
    for i,j in b:
        j=[]
        if i == 0:
            j.append(0)
            matrix.append(j)
        else:
            for k in range(i+1):
                if k == i:
                    j.append(0)
                else:
                    j.append(random.choice([0,1]))
            matrix.append(j)
            
    M = matrix        
    for i in range(len(M)):
        
        for j in range(1,len(M)-len(M[i])+1):
            M[i].append(M[i+j][i]) 
            
    return M

# Eliminate Loops 
def Deloop(x):
    l = len(x)

    for i in range(l):
        for j in range(i+1,l):
            j = l+i-j
            
            if j < len(x):          # list changes as loop is removed 
                if x[i] == x[j]:
                    # print("Delooping:  ", x)
                    [x.remove(k) for k in x[i+1:j+1]]
                    # print("Delooped to: ", x)
    return x

In [3]:
# Pheromone graph from CI book

SACOCI = [[0,0,0,1,0,1,0],
          [0,0,1,1,0,0,0],
          [0,1,0,1,1,0,0],
          [1,1,1,0,1,1,0],
          [0,0,1,1,0,0,1],
          [1,0,0,1,0,0,1],
          [0,0,0,0,1,1,0]]

In [4]:
# probability and selection using roulette wheel 

# Calculate cumulative sum for roulette wheel 
def PPi(pi):
    n = len(pi)
    ans = []
    
    for i in range(n):
        ans.append( sum([i for i in pi[:i+1]]) )
    return ans 

# Roulette Wheel 
def Roulette(ppi, Pop):
    n = len(ppi)
    ra = random.random()
    
    for i in range(n):
        if i == 0 :
            if ra > 0 and ra < ppi[i]:
                return Pop[i]
        else:
            if ra > ppi[i-1] and ra < ppi[i]:
                return Pop[i]

In [5]:
def Nexthob1(phermatrix, graph, preshob, prevhob = -1, alpha = 2):
    
    Cons = graph[preshob]  # obtain all connections for present hob
    if prevhob != -1:
        Cons[prevhob] = 0     # set previous connection to zero equivqlent to removal
    
    HP = [(phermatrix[preshob][i], i) for i in range(len(Cons)) if Cons[i] > 0 ] # get pheromone and corresponding hub
    #print(HP)
    
    if len(HP) == 0:    # if a dead end is met 
        # print("HEre")
        if prevhob == -1:
            print("No Transition Possible  ::::::")
        node = prevhob
    else:
        total = sum([i[0]**alpha for i in HP])  # total pheromone in connected hobs for probability calculation
        prob = [((i**alpha)/total,j) for i,j in HP] # get probability for corresponding hub
        node = Roulette(PPi([i[0] for i in prob]), [i[1] for i in prob]) # select hob with probability

    return node, preshob

def FindPath1(phermatrix, graph, takeoff, destination, prevhob = -1, alpha = 2):
    
    path = []
    path.append(takeoff)
    
    while takeoff != destination:
        
        takeoff, prevhob = Nexthob1(phermatrix, graph, takeoff, prevhob, alpha)
        path.append(takeoff)
        
    Deloop(path)
        
    return path

In [261]:
# Update Pheromone in visited arcs

def UpPheromone(route, matrix):
    
    tau = 1/len(route)
    for i in range(len(route)-1):
        
        matrix[route[i]][route[i+1]] += tau
        matrix[route[i+1]][route[i]] += tau

    return matrix

def UpPheromoneNPL(route, matrix):
    
    for i in range(len(route)-1):
        
        matrix[route[i]][route[i+1]] += 1
        matrix[route[i+1]][route[i]] += 1

    return matrix

# Evaporate pheromone 

def EvPheromone(matrix, rho):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if matrix[i][j] > 0:
                matrix[i][j] = (1-rho)*matrix[i][j]
            else:
                matrix[i][j] = 0
    return matrix
    

In [262]:
# Simple Ant Colony Optimiza
# Network graph from CI book
SACOCI = [[0,0,0,1,0,1,0],
          [0,0,1,1,0,0,0],
          [0,1,0,1,1,0,0],
          [1,1,1,0,1,1,0],
          [0,0,1,1,0,0,1],
          [1,0,0,1,0,0,1],
          [0,0,0,0,1,1,0]]

def ACO101(hobs, ants, starthob, destination, alpha = 2, runs = 50, pheromone = None, graph = None):
    
    Pher = FirstPheromone(hobs)
    Gr = Graph(hobs)
    #print(Gr)
    prevhob = -1
    Gr = SACOCI
    
    i = 0
    while i < runs:
        Ants = []
        #print(Gr)
        
        for j in range(ants):
            #print(j)
            stof = FindPath1(Pher, Gr, starthob, destination, prevhob, alpha)
            Ants.append(stof)
        #print(Gr)    
        
        for j in Ants:
            UpPheromoneNPL(j, Pher)
            
        i += 1 
        
    return np.array(Ants), np.array(Pher)

In [263]:
hobs = 7
ants = 500
S = 0
D = hobs-1
alpha = 3

S, D = 0, 4
runs = 100

ACO101(hobs, ants, S, D, alpha, runs)

(array([list([0, 5, 6, 4]), list([0, 5, 6, 4]), list([0, 5, 6, 4]),
        list([0, 5, 6, 4]), list([0, 5, 6, 4]), list([0, 5, 6, 4]),
        list([0, 5, 6, 4]), list([0, 5, 6, 4]), list([0, 5, 6, 4]),
        list([0, 5, 6, 4]), list([0, 5, 6, 4]), list([0, 5, 6, 4]),
        list([0, 5, 6, 4]), list([0, 5, 6, 4]), list([0, 5, 6, 4]),
        list([0, 5, 6, 4]), list([0, 5, 6, 4]), list([0, 5, 6, 4]),
        list([0, 5, 6, 4]), list([0, 5, 6, 4]), list([0, 5, 6, 4]),
        list([0, 5, 6, 4]), list([0, 5, 6, 4]), list([0, 5, 6, 4]),
        list([0, 5, 6, 4]), list([0, 5, 6, 4]), list([0, 5, 6, 4]),
        list([0, 5, 6, 4]), list([0, 5, 6, 4]), list([0, 5, 6, 4]),
        list([0, 5, 6, 4]), list([0, 5, 6, 4]), list([0, 5, 6, 4]),
        list([0, 5, 6, 4]), list([0, 5, 6, 4]), list([0, 5, 6, 4]),
        list([0, 5, 6, 4]), list([0, 5, 6, 4]), list([0, 5, 6, 4]),
        list([0, 5, 6, 4]), list([0, 5, 6, 4]), list([0, 5, 6, 4]),
        list([0, 5, 6, 4]), list([0, 5, 6, 4]), 

In [172]:
# Simple Ant Colony Optimiza with pheromone evaporation

def ACO102(hobs, ants, starthob, destination, alpha = 2, rho = 0.01, runs = 50, pheromone = None, graph = None):
    
    Pher = FirstPheromone(hobs)
    Gr = Graph(hobs)
    #print(Gr)
    prevhob = -1
    Gr = SACOCI
    
    i = 0
    while i < runs:
        Ants = []
        #print(Gr)
        
        for j in range(ants):
            #print(j)
            stof = FindPath1(Pher, Gr, starthob, destination, prevhob, alpha)
            Ants.append(stof)
        #print(Gr)
        
        Pher = EvPheromone(Pher, rho)
            
        for j in Ants:
            UpPheromone(j, Pher)
            
        i += 1 
        
    return np.array(Ants), np.array(Pher)

In [187]:
hobs = 7
ants = 128
S = 0
D = hobs-1
alpha = 1
rho = 0.2

S, D = 0, 4
runs = 100

ACO102(hobs, ants, S, D, alpha, rho, runs)

(array([[0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 